In [1]:
import os
from googletrans import Translator
import whisper
from pytube import YouTube
from pydub import AudioSegment
import cutlet
from lyrics_scrap import scrap
import pickle
from pathlib import Path

In [3]:
katsu = cutlet.Cutlet(use_foreign_spelling=False)
katsu.romaji("写真")

'Shashin'

In [10]:
song_info = [
    ('藍二乗', 'ヨルシカ'), 
    ('言って', 'ヨルシカ')
]
song_idx = 1

In [7]:
def save_mp3(video_url, folder_path, song_name):
    print(f"saving mp3 for {song_name}")
    yt = YouTube(video_url)
    aud = yt.streams.filter(only_audio=True).first()
    temp = aud.download(filename='temp_audio.mp3')
    audio = AudioSegment.from_file(temp)
    audio.export(f"{folder_path}/{song_name}.mp3", format='mp3')
    os.remove(temp)
    
def save_lyrics(song_name, song_composer, folderpath) -> str:
    print(f"saving lyrics for {song_name}")
    lyrics, yt_link = scrap(song_name, song_composer)
    with open(f"{folderpath}/{song_name}.pickle", 'wb') as fw:
        pickle.dump(lyrics, fw)
    return yt_link

targ_path = '../../mfvr/target'
file_names = [f.name for f in Path(targ_path).iterdir() if f.is_file()]
for name, composer in song_info:
    if f"{name}.pickle" not in file_names: 
        link = save_lyrics(name, composer, targ_path)
        save_mp3(link, targ_path, name)  
        
# with open(f"{targ_path}/{song_info[song_idx][0]}.pickle", 'rb') as fr:
#     ly = pickle.load(fr)

saving lyrics for 言って
saving mp3 for 言って


In [8]:
def rgbtxt(trgb=None, brgb=None, text="Default text"):
    tt = "" if trgb==None else f"\033[38;2;{trgb[0]};{trgb[1]};{trgb[2]}m"
    bt = "" if brgb==None else f"\033[48;2;{brgb[0]};{brgb[1]};{brgb[2]}m"
    return f"{tt}{bt}{text}\033[0m"

In [11]:
def whispering(audio_path, song_name):
    arg = dict(word_timestamps=True, condition_on_previous_text=False
               , language="Japanese"
               , initial_prompt=f"Song name is {song_name}.")
    model = whisper.load_model("medium")
    result = model.transcribe(audio_path, **arg)
    return result

rdict = whispering(f"{targ_path}/{song_info[song_idx][0]}.mp3", song_info[song_idx][0])

c:\Users\naniri\anaconda3\envs\vocal\lib\site-packages\whisper\timing.py:42: UserWarning: Failed to launch Triton kernels, likely due to missing CUDA toolkit; falling back to a slower median kernel implementation...
  warnings.warn(
c:\Users\naniri\anaconda3\envs\vocal\lib\site-packages\whisper\timing.py:146: UserWarning: Failed to launch Triton kernels, likely due to missing CUDA toolkit; falling back to a slower DTW implementation...
  warnings.warn(
c:\Users\naniri\anaconda3\envs\vocal\lib\site-packages\whisper\timing.py:42: UserWarning: Failed to launch Triton kernels, likely due to missing CUDA toolkit; falling back to a slower median kernel implementation...
  warnings.warn(
c:\Users\naniri\anaconda3\envs\vocal\lib\site-packages\whisper\timing.py:146: UserWarning: Failed to launch Triton kernels, likely due to missing CUDA toolkit; falling back to a slower DTW implementation...
  warnings.warn(


In [ ]:
rdict['segments']

In [12]:
def text_to_romaji(lyrics):
    katsu = cutlet.Cutlet(use_foreign_spelling=False); l = []
    for phrase in lyrics:
        l.append(katsu.romaji(phrase['text']))
    return l

romaji_list = text_to_romaji(rdict['segments'])
print(romaji_list)

['Ano ne watakushi jitsu wa kizutsuiteru n dakedo', '', '', '', 'Horakun ga itta koto', 'Amai', 'Kangaetai to omoenakute', 'Wasureteta n dakedo', 'Moumokuteki ni mousouteki ni mousouteki ni ikite', 'Shoudouteki na shousouteki na shoukyokuteki na mama ja dame datta n da', 'Kitto', 'Jinsei saikou no hi wo mae ni omou no darou', 'Zenbu zenbu itai no desukedo', 'Aa, itsu ka', 'Jinsei saikou no hi', 'Kimi ga inai koto wo', 'Motto motto motto', 'Motto chanto itte', '[uta]', 'Ano ne', 'Sora ga aoi no tte dou yatte', 'Tsutaereba ii n darou ne', 'Yoru no kumo ga takai no tte dou sureba', 'Kimi mo wakaru n darou itte', 'Ano ne watakushi jitsu wa wakatteru no mou kimi ga itta koto', 'Ano ne wakarazu yatte iu n darou ne wasuretai n dakedo', 'Motto chanto itte yo wasurenai yo memo ni shite yo', 'Asu 10 ji ni hoomu de machiawase to ka shiyou', 'Utau wa jitte mo hana da', 'Natsu ga satte mo suiboku wa setsu na', 'Kuchi ni dashite', 'Koi ni dashite', 'Kimi ga itte', 'Soshite jinsei saikou no hi', 'Kim

In [13]:
for idx, seg in enumerate(rdict['segments']):
    for word in seg['words']:
        p = int(255*float(word['probability']))
        print(rgbtxt((0,0,0),(255,p,p),word['word']), end='')
    print()
    print(romaji_list[idx])

あのね 私 実は傷ついてるんだけど
Ano ne watakushi jitsu wa kizutsuiteru n dakedo






ほら君が言ったこと
Horakun ga itta koto
甘い
Amai
考えたいと思えなくて
Kangaetai to omoenakute
忘れてたんだけど
Wasureteta n dakedo
盲目的に 妄想的に 妄想的に 生きて
Moumokuteki ni mousouteki ni mousouteki ni ikite
衝動的な 焦燥的な 消極的なままじゃ ダメだったんだ
Shoudouteki na shousouteki na shoukyokuteki na mama ja dame datta n da
きっと
Kitto
人生最高の日を前に思うのだろう
Jinsei saikou no hi wo mae ni omou no darou
全部全部痛いのですけど
Zenbu zenbu itai no desukedo
ああ、いつか
Aa, itsu ka
人生最高の日
Jinsei saikou no hi
君がいないことを
Kimi ga inai koto wo
もっともっともっと
Motto motto motto
もっとちゃんと言って
Motto chanto itte
【歌】
[uta]
あのね
Ano ne
空が青いのってどうやって
Sora ga aoi no tte dou yatte
伝えればいいんだろうね
Tsutaereba ii n darou ne
夜の 雲が 高いのってどうすれば
Yoru no kumo ga takai no tte dou sureba
君もわかるんだろう 言って
Kimi mo wakaru n darou itte
あのね 私 実は分かってるの もう君が言ったこと
Ano ne watakushi jitsu wa wakatteru no mou kimi ga itta koto
あのね わからずやって言うんだろうね 忘れたいんだけど
Ano ne wakarazu yatte iu n darou ne wasuretai n dakedo
もっとちゃんと言ってよ 忘れないよメモにしてよ
Motto chanto itte yo wa